# First Agent

In [1]:
from langchain_ollama import ChatOllama
from IPython.display import display, Markdown

In [2]:
def print_md(response):
    display(Markdown(response.content))

In [3]:
# model_name = "phi"
# model_name = "qwen3-vl:2b-thinking"
model_name = "qwen3-vl:2b-instruct" # supports tools
# model_name = "llama3.2"
# model_name = "granite4"
# model_name = "gemma3" # does not support tools
# model_name = "granite3.3"
# model_name = "llama3.1"
# model_name = "gpt-oss"

In [4]:
model = ChatOllama(
    model=model_name,
    temperature=0
)

In [ ]:
response = model.invoke("What is LangChain?")
print_md(response)

## Messages

There are different message types:,
- **SystemMessage** - Instructions for how the model should behave,
- **HumanMessage** - User input,
- **AIMessage** - Model responses,
- **ToolMessage** - Results from tool executions

In [ ]:
from langchain.messages import HumanMessage, SystemMessage

In [ ]:
messages = [
    SystemMessage(content="You are a helpful AI assistant that explains technical concepts simply."),
    HumanMessage(content="What is an agent?"),
]

response = model.invoke(messages)
print_md(response)

## Multi-turn Conversations

In [ ]:
# Continue the conversation
messages.append(response)  # Add AI response to history
messages.append(HumanMessage(content="Can you give me an example?"))

response = model.invoke(messages)
print_md(response)

## Tools

In [5]:
import requests
import json

from langchain_core.tools import tool

In [6]:
@tool
def get_weather(latitude: float, longitude: float) -> str:
    """Get current temperature in Fahrenheit and weather code for given coordinates.

    Args:
        latitude: Latitude coordinate
        longitude: Longitude coordinate

    Returns:
        JSON string with temperature_fahrenheit and weather_code (do not include the code in your response, translate it to plain English)
    """
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current": "temperature_2m,weather_code",
        "temperature_unit": "fahrenheit"
    }

    weather = requests.get(url, params=params).json()["current"]
    temperature = weather["temperature_2m"]
    weather_code = weather["weather_code"]

    result = {
        "temperature_fahrenheit": temperature,
        "weather_code": weather_code
    }

    return json.dumps(result)

In [7]:
# Test a tool directly with SF's coordinates
print(get_weather.invoke({"latitude": 37.77, "longitude": 122.42}))

{"temperature_fahrenheit": 23.3, "weather_code": 3}


In [8]:
tools = [get_weather]
model = model.bind_tools(tools)

In [9]:
message = "What's the weather like in Seattle? (Seattle's coordinates are approximately 47.6° N latitude and 122.33° W longitude) "

response = model.invoke(message)

print("Tool calls:", response.tool_calls)

Tool calls: [{'name': 'get_weather', 'args': {'latitude': 47.6, 'longitude': -122.33}, 'id': '7da2441e-9f11-4656-a57f-6e305144db0a', 'type': 'tool_call'}]
